In [1]:
#default_exp oo_hp

In [2]:
#export
import fastai2
from fastai2.tabular.all import *
from fastai2.metrics import *

In [3]:
#export
from copy import deepcopy

In [4]:
#export
import optuna

In [5]:
#export
SEED = 42

In [6]:
from nbdev import export

In [7]:
from nbdev.export import *

In [8]:
#export
from collections.abc import Iterable

# HyPSTER Classes

In [9]:
#export
from inspect import signature, Parameter
import functools

def auto_assign(func):
    # Signature:
    sig = signature(func)
    for name, param in sig.parameters.items():
        if param.kind in (Parameter.VAR_POSITIONAL, Parameter.VAR_KEYWORD):
            raise RuntimeError('Unable to auto assign if *args or **kwargs in signature.')
    # Wrapper:
    @functools.wraps(func)
    def wrapper(self, *args, **kwargs):
        for i, (name, param) in enumerate(sig.parameters.items()):
            # Skip 'self' param:
            if i == 0: continue
            # Search value in args, kwargs or defaults:
            if i - 1 < len(args):
                val = args[i - 1]
            elif name in kwargs:
                val = kwargs[name]
            else:
                val = param.default
            setattr(self, name, val)
        func(self, *args, **kwargs)
    return wrapper

In [10]:
#export
class HypsterBase:
    def __init__(self): return
#TODO: add stuff here

In [11]:
#export
class HpExpression:
    def __init__(self, exp1, exp2):
        self.exp1 = exp1; self.exp2 = exp2
    
    def sample(self, trial): raise NotImplementedError
    
    #TODO: check what to do when name=None
    def get_name(self):
        if self.name is not None: return self.name

        name = ""
        if self.exp1 is not None and isinstance(self.exp1, HpExpression) and hasattr(self.exp1, "name"):
            name += self.exp1.name
        if self.exp2 is not None and isinstance(self.exp2, HpExpression) and hasattr(self.exp2, "name"):
            if len(name) > 0:
                name +=  "_"
            name += self.exp2.name
        self.name = name
        return self.name
        #TODO: refactor

    def __add__(self, other):  return AddExpression(self, other)
    def __radd__(self, other): return AddExpression(other, self)
    def __sub__(self, other):  return SubExpression(self, other)
    def __rsub__(self, other): return SubExpression(other, self)
    def __mul__(self, other):  return MulExpression(self, other)
    def __rmul__(self, other): return MulExpression(other, self)
    def __div__(self, other):  return DivExpression(self, other)
    def __rdiv__(self, other): return DivExpression(other, self)
    def __pow__(self, other):  return PowExpression(self, other)
    def __rpow__(self, other): return PowExpression(other, self)

In [12]:
#export
HYPSTER_TYPES = (HypsterBase, HpExpression)

In [13]:
#export
class SubExpression(HpExpression):
    def sample(self, trial):
        exp1 = sample_hp(self.exp1, trial)
        exp2 = sample_hp(self.exp2, trial)
        return exp1 - exp2

In [14]:
#export
class AddExpression(HpExpression):    
    def sample(self, trial):
        exp1 = sample_hp(self.exp1, trial)
        exp2 = sample_hp(self.exp2, trial)
        return exp1 + exp2

In [15]:
#export
class MulExpression(HpExpression):    
    def sample(self, trial):
        exp1 = sample_hp(self.exp1, trial)
        exp2 = sample_hp(self.exp2, trial)
        return exp1 * exp2

In [16]:
#export
class DivExpression(HpExpression):    
    def sample(self, trial):
        exp1 = sample_hp(self.exp1, trial)
        exp2 = sample_hp(self.exp2, trial)
        return exp1 / exp2

In [17]:
#export
class PowExpression(HpExpression):    
    def sample(self, trial):
        exp1 = sample_hp(self.exp1, trial)
        exp2 = sample_hp(self.exp2, trial)
        return exp1 ** exp2

In [18]:
#TODO: add round, int, floor etc...
#TODO: add Brackets (), [] etc...?

In [19]:
def foo(a, b="hello"):
    print(a)

# Atomic HPs

In [20]:
#export
def set_init_name(default_name, name):
    if name is not None:
        return name, True
    else:
        return default_name, False

In [21]:
def remove_prefix(text, prefix):
    if text.startswith(prefix):
        return text[len(prefix):]
    return text

In [22]:
def get_name_from_hp(hp):
    return remove_prefix(hp.__class__.__name__.lower(), "hp")

In [23]:
# upon init -> assign a name by manual name or type
# upon using prepare - if there isn't a manual name - assign the name by arg_name

In [24]:
# upon initializing a study, recursively go through arguments and check if there are same expressions.
# for each arg in args:
    # if there is another name and both manually defined -- error
    # if there is another name and both and not manually defined --> add suffix to both by order
    # if one is not manually defined --> get serial number of current and add suffix only to the one not manually defined

In [25]:
#export
def get_name(hp):
    if isinstance(hp, HpExpression):
        return hp.name
    if hasattr(hp, "call"): #TODO: isinstance(hp, HypsterPrepare):
        #TODO: check if exists(?)
        return get_name(hp.call)
    #TODO: check this part. why does norm=Normalize() return _TfmMeta?
    if hasattr(hp, "__name__"):
        return hp.__name__
    if hasattr(hp, "__class__") and hasattr(hp.__class__, "__name__"):
        return hp.__class__.__name__
    if callable(hp):
        #uninstantiated function/class
        return hp.__name__
    else:
        print("Error! can't get name!")
        return

In [26]:
class Shhhh:
    def __init__(self): return

In [27]:
def foo(x=5):
    print(x)

In [28]:
a = Shhhh()

In [29]:
b=5

In [30]:
b.__class__.__name__

'int'

In [31]:
b = 5

In [32]:
type(b)

int

In [33]:
type(a)

__main__.Shhhh

In [34]:
callable(a)

False

In [35]:
a.__class__.__name__

'Shhhh'

In [36]:
#export
class HpAtomic(HpExpression):
    def __init__(self):
        #...

In [37]:
#export
class HpInt(HpAtomic):
    @auto_assign
    def __init__(self, low, high, step=1, name=None):
        self.name, self.manual_name = set_init_name("int", name)
        super().__init__(...)
    
    def sample(self, trial):
        return trial.suggest_int(self.name, self.low, self.high, self.step)

In [38]:
#export
class HpFloat(HpAtomic):
    @auto_assign
    def __init__(self, low, high, log=False, step=None, name=None):
        self.name, self.manual_name = set_init_name("float", name)
        #TODO: check what's up with log and step
        #TODO: move result to HpExpression?
    
    def sample(self, trial):
        return trial.suggest_float(self.name, self.low, self.high)
    
    #TODO: warn if log=True & step is not None
    #TODO: check what is the "*" in the function definition

In [39]:
#export
def _log_optuna_param(param_name, result, trial):
    trial.set_user_attr(param_name, result)

In [40]:
#export
class HpCategorical(HpAtomic):
    @auto_assign
    def __init__(self, choices, name=None):
        self.name, self.manual_name = set_init_name("category", name)

    def sample(self, trial):         
        choices           = self.choices
        name              = self.name
        optuna_valid_cats = [str, int, float, bool, NoneType] #TODO: add more + move to global area
        
        if any([type(choice) not in optuna_valid_cats for choice in self.choices]):
            #TODO: add check for "choice.__name__"
            self.items_str = [choice.__name__ for choice in choices]
            self.str_dict  = dict(zip(self.items_str, choices))
            chosen_hp      = trial.suggest_categorical(name, self.str_dict)
            result         = self.str_dict[chosen_hp]
        else:
            result = trial.suggest_categorical(name, choices)
        return result

In [41]:
#export
class HpBool(HpCategorical):
    def __init__(self, name=None):
        super().__init__(choices=[False, True])
        self.name, self.manual_name = set_init_name("boolean", name)

In [42]:
#export
class HpWrapper(HpExpression):
    def __init__(self): return
    def get_inner_obj(self): return

In [43]:
#export
class HpFunc(HpWrapper):
    def __init__(self, func, name=None, **kwargs):
        self.name, self.manual_name = set_init_name("func", name)
        self.func = func
        self.kwargs = kwargs
    
    def get_inner_obj(self): return self.func
    
    def sample(self, trial):
        result = self.func(trial, **self.kwargs)
        _log_optuna_param(self.name, result, trial)
        return result

In [44]:
#export
class HpToggle(HpWrapper):
    @auto_assign
    def __init__(self, hp, name=None):
        if isinstance(hp, HypsterPrepare):
            #TODO: go over args and kwargs and add prefix
            #hp.name = f"{get_name(hp)}_{hp.name}"
        inner_name = get_name(hp)
        self.name, self.manual_name = set_init_name(f"toggle_{inner_name}", name)
    def get_inner_obj(self): return self.hp
    def sample(self, trial): return HpBool(self.name).sample(trial)

In [45]:
#export
class HpVarLenIterable(HpWrapper):
    def __init__(self): return

In [46]:
#export
class HpVarLenList(HpVarLenIterable):
    #TODO: think of a better name?
    @auto_assign
    def __init__(self, min_len, max_len, hp, same_value=False, name=None):
        self.name, self.manual_name = set_init_name("var_len_list", name)
    
    def sample(self, trial):
        lst_len = trial.suggest_int(self.name, self.min_len, self.max_len)
        lst = []
        if (self.same_value) or (not contains_hypster(self.hp, HYPSTER_TYPES)):
            lst = [sample_hp(self.hp, trial)] * lst_len
        else:
            for i in range(lst_len):
                hp = deepcopy(self.hp)
                hp.result = None
                hp.name = f"{hp.get_name()}_{i+1}"
                result = sample_hp(hp, trial)
                lst.append(result)

        return lst

In [47]:
#export
class HpVarLenTuple(HpVarLenIterable):
    #TODO: think of a better name?
    @auto_assign
    def __init__(self, min_len, max_len, hp, same_value=False, name=None):
        self.name, self.manual_name = set_init_name("var_len_tuple", name)
    
    def sample(self, trial):
        lst_len = trial.suggest_int(self.name, self.min_len, self.max_len)
        lst = []
        if (self.same_value) or (not contains_hypster(self.hp, HYPSTER_TYPES)):
            lst = [sample_hp(self.hp, trial)] * lst_len
        else:
            for i in range(lst_len):
                hp = deepcopy(self.hp)
                hp.result = None
                hp.name = f"{hp.get_name()}_{i+1}"
                result = sample_hp(hp, trial)
                lst.append(result)

        return list_to_tuple(lst)
    #TODO: refactor!!!

In [48]:
#export
class HpIterable(HpWrapper):
    def __init__(self): return

In [49]:
#export
def list_to_tuple(lst): return (*lst, )

In [50]:
#export
def populate_iterable(iterable, trial):
    sampled_lst = []
    for item in iterable:
        if isinstance(item, HpToggle):
            if sample_hp(item, trial):
                sampled_lst.append(sample_hp(item.hp, trial))
        else:
            sampled_lst.append(sample_hp(item, trial))
    return sampled_lst

In [51]:
#export
def populate_dict(dct, trial):
    sampled_dict = {}
    for key, value in dct.items():
        if isinstance(value, HpToggle):
            if sample_hp(value, trial):
                sampled_dict[key] = sample_hp(value.hp, trial)
        else:
            sampled_dict[key] = sample_hp(value, trial)
    return sampled_dict

In [52]:
#export
class HpList(HpIterable):
    @auto_assign
    def __init__(self, lst):
        self.name = "list"
    def get_inner_obj(self): return self.lst
    def sample(self, trial):
        return populate_iterable(self.lst, trial)

In [53]:
#export
class HpTuple(HpIterable):
    @auto_assign
    def __init__(self, tup):
        self.name = "tuple"
    def get_inner_obj(self): return self.tup
    def sample(self, trial):
        return list_to_tuple(populate_iterable(self.tup, trial))

In [54]:
#export
class HpDict(HpIterable):
    @auto_assign
    def __init__(self, dct):
        self.name = "dict"
    def get_inner_obj(self): return self.dct
    def sample(self, trial):
        return populate_dict(self.dct, trial)

# Tests

In [55]:
#export
DATA_STRUCTURES = (set, list, tuple, dict)

In [56]:
#export
def contains_hypster(x, types):
    if not isinstance(x, DATA_STRUCTURES):
        x = [x]
    elif isinstance(x, dict):
        x = x.values()
        
    for item in x:
        if isinstance(item, DATA_STRUCTURES):
            if contains_hypster(item, types):
                return True
        else:
            if isinstance(item, types):
                return True
    return False

In [57]:
#TODO!: check if class attributes / methods? have hypster in them

## Test contains_hypster

In [58]:
hps = []

In [59]:
hps.append(HpInt(2, 10))

In [60]:
hps.append({"hi" : 4, "hello": HpFloat(1.0, 2.0)})

In [61]:
hps.append(["a", 2, HpInt(1, 19)])

In [62]:
hps.append(["a", 2, {"ho": "hello"}, {"hi" : HpCategorical([1,2,3])}])

In [63]:
[contains_hypster(hp, HpExpression) for hp in hps]

[True, True, True, True]

In [64]:
#export
def sample_hp(hp, trial):
    if not contains_hypster(hp, HYPSTER_TYPES):
        return hp
    #TODO: change to dynamic dispatch
    if isinstance(hp, list):
        hp = HpList(hp)
    elif isinstance(hp, tuple):
        hp = HpTuple(hp)
    elif isinstance(hp, dict):
        hp = HpDict(hp)
    return hp.sample(trial)
#TODO: handle list of lists
#TODO!: check if class attributes / methods? have hypster in them

In [65]:
#export
def run_hp_test(hp):
    def objective(trial):
        print(sample_hp(hp, trial))
        return 1.0        
    
    optuna.logging.set_verbosity(0)
    pruner = optuna.pruners.NopPruner()
    study = optuna.create_study(direction="maximize", pruner=pruner)
    study.optimize(objective, n_trials=5, timeout=600)
    return study

### HpToggle

In [66]:
x = [1, 5, HpToggle(10)]

10
<class 'int'>


In [67]:
study = run_hp_test(x)

[1, 5]
[1, 5, 10]
[1, 5]
[1, 5]
[1, 5, 10]


In [68]:
study.best_params

{'toggle_int': False}

In [69]:
y = (1, 5, HpToggle(10))

10
<class 'int'>


In [70]:
study = run_hp_test(y)

(1, 5, 10)
(1, 5, 10)
(1, 5, 10)
(1, 5, 10)
(1, 5, 10)


In [71]:
study.best_params

{'toggle_int': True}

In [72]:
z = {"hi" : 1, "hello" : 5, "howdy" : HpToggle(10)}

10
<class 'int'>


In [73]:
study = run_hp_test(z)

{'hi': 1, 'hello': 5}
{'hi': 1, 'hello': 5}
{'hi': 1, 'hello': 5, 'howdy': 10}
{'hi': 1, 'hello': 5, 'howdy': 10}
{'hi': 1, 'hello': 5, 'howdy': 10}


In [74]:
study.best_params

{'toggle_int': False}

### Int

In [75]:
x = HpInt(2, 10)
x_lst = [HpInt(2, 10, name="int_1"), HpInt(12,29, name="int_2")]
y = HpInt(50, 300, 50)

In [76]:
study = run_hp_test(x)

7
7
10
10
9


In [77]:
study.best_params

{'int': 7}

In [78]:
study = run_hp_test(x_lst)

[6, 20]
[7, 29]
[10, 28]
[9, 18]
[7, 16]


In [79]:
study.best_params

{'int_1': 6, 'int_2': 20}

In [80]:
study = run_hp_test(y)

100
150
250
200
100


In [81]:
study.best_params

{'int': 100}

In [82]:
z = HpVarLenList(1, 5, x, same_value=False)

In [83]:
study = run_hp_test(z)

[4]
[5]
[9, 9]
[2, 4, 7, 6, 6]
[10, 5]


In [84]:
study.best_params

{'var_len_list': 1, 'int_1': 4}

### Float

In [85]:
x = HpFloat(1.0, 2.0)
y = HpVarLenList(1, 5, x, same_value=False)

In [86]:
study = run_hp_test(x)

1.3598158822684911
1.361613213666068
1.127737997874184
1.18726520557275
1.2849818251741536


In [87]:
study.best_params

{'float': 1.3598158822684911}

In [88]:
study = run_hp_test(y)

[1.4052558612112116, 1.8002345896463074, 1.2309049073985185]
[1.9024990000922348]
[1.9052069035429235, 1.2417053102873061, 1.5629144704711528, 1.9000842486027345]
[1.7948025702279493, 1.9114660524048914, 1.0593887948346339, 1.23773788443582, 1.83112996609706]
[1.9033949267643016, 1.6115811275558425]


In [89]:
study.best_params

{'var_len_list': 3,
 'float_1': 1.4052558612112116,
 'float_2': 1.8002345896463074,
 'float_3': 1.2309049073985185}

### Categorical

In [90]:
x = HpCategorical(["cat", "meow", "rrrrrr"])
y = HpCategorical([Adam, SGD, QHAdam])
z = HpVarLenList(1, 5, x, same_value=False)

In [91]:
study = run_hp_test(x)

rrrrrr
cat
cat
rrrrrr
meow


In [92]:
study.best_params

{'category': 'rrrrrr'}

In [93]:
study = run_hp_test(y)

<function Adam at 0x000001B6A4719400>
<function Adam at 0x000001B6A4719400>
<function Adam at 0x000001B6A4719400>
<function Adam at 0x000001B6A4719400>
<function QHAdam at 0x000001B6A4719730>


In [94]:
study.best_params

{'category': 'Adam'}

In [95]:
study = run_hp_test(z)

['cat', 'rrrrrr', 'rrrrrr', 'meow']
['meow']
['rrrrrr', 'cat', 'rrrrrr']
['cat', 'rrrrrr', 'cat']
['cat']


In [96]:
study.best_params

{'var_len_list': 4,
 'category_1': 'cat',
 'category_2': 'rrrrrr',
 'category_3': 'rrrrrr',
 'category_4': 'meow'}

### Boolean

In [97]:
x = HpBool()
y = HpVarLenList(1, 5, x, same_value=False)

In [98]:
run_hp_test(x)

False
True
False
True
False


In [99]:
run_hp_test(y)

[False, True, True]
[False]
[False, True, True]
[True, False, False, False]
[True, False]


### HpIterable

In [100]:
x = HpBool()

In [101]:
lst = [x, 5]
tup = [x, 5]
dct = {"first" : x, "second" : 5}

In [102]:
start_mom = HpFloat(0.85, 0.99)

In [103]:
mom = (start_mom, start_mom - 0.1, 4.0)

In [104]:
study = run_hp_test(mom)

(0.8575236780638439, 0.7575236780638439, 4.0)
(0.8627229789395604, 0.7627229789395604, 4.0)
(0.8774074710413755, 0.7774074710413755, 4.0)
(0.8739017652202512, 0.7739017652202512, 4.0)
(0.8665715626399638, 0.7665715626399638, 4.0)


In [105]:
study.best_params

{'float': 0.8575236780638439}

In [106]:
study = run_hp_test(lst)

[True, 5]
[True, 5]
[False, 5]
[True, 5]
[False, 5]


In [107]:
study.best_params

{'boolean': True}

In [108]:
study = run_hp_test(tup)

[True, 5]
[True, 5]
[False, 5]
[True, 5]
[False, 5]


In [109]:
study.best_params

{'boolean': True}

In [110]:
study = run_hp_test(dct)

{'first': False, 'second': 5}
{'first': False, 'second': 5}
{'first': False, 'second': 5}
{'first': False, 'second': 5}
{'first': False, 'second': 5}


In [111]:
study.best_params

{'boolean': False}

In [112]:
#TODO: support expressions like HpInt(...) * HpFloat(...) ?

In [113]:
notebook2script()

Converted 00_core.ipynb.
Converted 01_api.ipynb.
Converted 02_oo_hp.ipynb.
Converted 03_hypster_prepare.ipynb.
Converted 04_tabular_api.ipynb.
Converted 05_sklearn.ipynb.
Converted fastai_adult_tutorial.ipynb.
Converted index.ipynb.
